In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('CGI Merchant Debt Woes Map - Sheet1.csv',dtype='str')

In [5]:
df = df.fillna('Not Available')

## Image URL Collector

In [5]:
df.columns

Index(['Property', 'Google Coordinates', 'Caption ', 'Story link ',
       'image_links'],
      dtype='object')

In [7]:
df['image_links'] = ''
links = []
for url in df['Story link ']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find('img',{'class':'MainArtImage_image__2YtGm'})
        images = images['src']
        links.append(images)
    except:
        links.append('no_image')

df['image_links'] = links

In [9]:
# print(df['image_links'].iloc[0])

In [9]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
df['image_links'].iloc[-1]

'no_image'

In [11]:
# df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [14]:
df['description'] = df['Caption ']

In [18]:
df.columns

Index(['Property', 'Google Coordinates', 'Caption ', 'Story link ',
       'image_links', 'Description', 'description', 'short_description',
       'remaining_desc'],
      dtype='object')

In [19]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["Story link "] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0                                                  NaN
1                                                  NaN
2                                                  NaN
3                                                  NaN
4                                                  NaN
5                                                  NaN
6                                                  NaN
7    <a href="https://therealdeal.com/miami/2024/12...
8    <a href="https://therealdeal.com/miami/2024/05...
9    <a href="https://therealdeal.com/miami/2024/08...
Name: description_link, dtype: object

In [30]:
def popup_html(row):
    i = row
    property_ = df['Property'].iloc[i]
    caption = df['Caption '].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <h3>{}'''.format(property_) + '''</h3><br>
    {}'''.format(caption) + '''<br>
    </html>
    '''
    return html

In [31]:
df.columns

Index(['Property', 'Google Coordinates', 'Caption ', 'Story link ',
       'image_links', 'Description', 'description', 'short_description',
       'remaining_desc', 'description_link', 'lat', 'lon'],
      dtype='object')

In [32]:
# Split the string "lat, lon" into two columns
df[["lat", "lon"]] = df["Google Coordinates"].str.split(",", expand=True)

# Convert the new columns to float
df["lat"] = df["lat"].astype(float)
df["lon"] = df["lon"].astype(float)

df

,Property,Google Coordinates,Caption,Story link,image_links,Description,description,short_description,remaining_desc,description_link,lat,lon
0,Nexus portfolio Vista Park,"26.7151168324441, -80.1510386463918",CGI Merchant Group's portfolio includes co-wor...,NaN,no_image,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's,portfolio includes co-working provider Nexus ...,NaN,26.715117,-80.151039
1,Nexus portfolio (Wellington),"26.61868561289822, -80.17787976971158",CGI Merchant Group's portfolio includes co-wor...,NaN,no_image,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's,portfolio includes co-working provider Nexus ...,NaN,26.618686,-80.177880
2,Nexus portfolo (Stuart),"27.215397640808465, -80.26225193781245",CGI Merchant Group's portfolio includes co-wor...,NaN,no_image,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's,portfolio includes co-working provider Nexus ...,NaN,27.215398,-80.262252
3,Nexus (Boca Raton),"26.373733435849157, -80.20500499300631",CGI Merchant Group's portfolio includes co-wor...,NaN,no_image,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's,portfolio includes co-working provider Nexus ...,NaN,26.373733,-80.205005
4,Nexus (Boynton Beach),"26.547463227846205, -80.08487914444163",CGI Merchant Group's portfolio includes co-wor...,NaN,no_image,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's portfolio includes co-wor...,CGI Merchant Group's,portfolio includes co-working provider Nexus ...,NaN,26.547463,-80.084879
5,55 Miracle Mile,"25.750506885954568, -80.25618177826915","The $25 million loan, which now has a balance ...",NaN,no_image,"The $25 million loan, which now has a balance ...","The $25 million loan, which now has a balance ...",The $25 million,"loan, which now has a balance of $24.2 millio...",NaN,25.750507,-80.256182
6,3480 Main Highway,"25.72661589178331, -80.24424767323595",The latest trouble for CGI Merchant came this ...,NaN,no_image,The latest trouble for CGI Merchant came this ...,The latest trouble for CGI Merchant came this ...,The latest trouble,for CGI Merchant came this week when Madison ...,NaN,25.726616,-80.244248
7,550 Biltmore Way,"25.747882663916485, -80.26601487078878","In Coral Gables, lender Cerberus Capital Manag...",https://therealdeal.com/miami/2024/12/05/raoul...,no_image,"In Coral Gables, lender Cerberus Capital Manag...","In Coral Gables, lender Cerberus Capital Manag...","In Coral Gables,",lender Cerberus Capital Management last month...,"<a href=""https://therealdeal.com/miami/2024/12...",25.747883,-80.266015
8,Gabriel South Beach,"25.776535129022726, -80.1318352747398",Deutsche Bank filed a UCC foreclosure on the 1...,https://therealdeal.com/miami/2024/05/28/cgi-m...,no_image,Deutsche Bank filed a UCC foreclosure on the 1...,Deutsche Bank filed a UCC foreclosure on the 1...,Deutsche Bank filed,a UCC foreclosure on the 132-key Gabriel Sout...,"<a href=""https://therealdeal.com/miami/2024/05...",25.776535,-80.131835
9,Gabriel Downtown Miami,"25.785179642550002, -80.18998275907768","On Aug. 5, Madison Realty Capital took back th...",https://therealdeal.com/miami/2024/08/09/cgi-m...,no_image,"On Aug. 5, Madison Realty Capital took back th...","On Aug. 5, Madison Realty Capital took back th...","On Aug. 5, Madison",Realty Capital took back the 129-key hotel at...,"<a href=""https://therealdeal.com/miami/2024/08...",25.785180,-80.189983


In [33]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'CGI Merchant Debt Woes')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True),max_width=400)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [34]:
m.save('index.html')

## Map URL Snagger

In [35]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFla_CGI_Merchant_Debt_Woes
